In [18]:
import redis 
import unittest
import bisect
import uuid

In [19]:
valid_characters = '`abcdefghijklmnopqrstuvwxyz{'

def find_prefix_range(prefix):
    posn = bisect.bisect_left(valid_characters, prefix[-1:])
    suffix = valid_characters[(posn or 1) - 1]
    return f'{prefix[:-1]}{suffix}{{', f'{prefix}{{'

In [34]:
def autocomplete_on_prefix(conn, guild, prefix):
    start, end = find_prefix_range(prefix)
    identifier = str(uuid.uuid4())
    
    start += identifier
    end += identifier
    
    zset_name = f'members:{guild}'
    
    # Add the start/end range items to the zset.
    conn.zadd(zset_name, {start: 0, end: 0})
    pipeline = conn.pipeline(True)
    while 1:
        try:
            # Find the ranks of our end points.
            pipeline.watch(zset_name)
            sindex = pipeline.zrank(zset_name, start)
            eindex = pipeline.zrank(zset_name, end)
            erange = min(sindex + 9, eindex - 2)
            pipeline.multi()
            
            # Get the values inside our range, and cleanup.
            pipeline.zrem(zset_name, start, end)
            pipeline.zrange(zset_name, sindex, erange)
            items = pipeline.execute()[-1]
            break
        except redis.exceptions.WatchError:
            # Retry if someone modified our autocomplete zset.
            continue
    # Remove start/end entries if an autocomplete was in progress.
    return [item for item in items if '{' not in item]

In [35]:
def join_guild(conn, guild, user):
    conn.zadd(f'members:{guild}', {user: 0})

In [41]:
def leave_guild(conn, guild, user):
    conn.zrem(f'members:{guild}', user)

In [43]:
class TestAddressBookAutocomplete(unittest.TestCase):
    def setUp(self):
        self.conn = redis.Redis(password='123456', decode_responses=True)
    
    def tearDown(self):
        self.conn.close()
        
    def test_sum(self):
        self.assertEqual(1+1, 2)
        
    def test_autocomplete(self):
        join_guild(self.conn, 'greenhouse', 'alpha')
        join_guild(self.conn, 'greenhouse', 'alice')
        join_guild(self.conn, 'greenhouse', 'bob')
        
        expected = ['alpha', 'alice']
        actual = autocomplete_on_prefix(self.conn, 'greenhouse', 'al')
        self.assertListEqual(sorted(actual), sorted(expected))
        
        leave_guild(self.conn, 'greenhouse', 'alpha')
        expected = ['alice']
        actual = autocomplete_on_prefix(self.conn, 'greenhouse', 'al')
        self.assertListEqual(actual, expected)
        
        expected = ['bob']
        actual = autocomplete_on_prefix(self.conn, 'greenhouse', 'b')
        self.assertListEqual(actual, expected)
        
if __name__ == '__main__':
    unittest.main(argv=['excluded'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.051s

OK


In [16]:
find_prefix_range('hello')

posn 15
suffix n


('helln{', 'hello{')